1.Voting

In [229]:
using JuMP, Gurobi

#voters information, first row Republicans, second row Democrats
voters = [80 60 40 20 40 40 70 50 70 70;
          34 44 44 24 114 64 14 44 54 64]

m = Model(solver=GurobiSolver(OutputFlag=0))

#binary variables on which congressional representative is going to be on
@variable(m, z[1:10, 1:5], Bin)
#binary variables on which 
@variable(m, majority[1:5], Bin)

#each city can only be in one congress
for i in 1:10
    @constraint(m, sum(z[i,j] for j in 1:5) == 1)
end

for j in 1:5
    @constraint(m, sum(voters[1, :] .* z[:, j]) + 
        sum(voters[2, :] .* z[:, j]) <= 250)
    @constraint(m, sum(voters[1, :] .* z[:, j]) + 
        sum(voters[2, :] .* z[:, j]) >= 150)
end

for j in 1:5
    @constraint(m, sum(voters[1, :] .* z[:, j]) - 
        sum(voters[2, :] .* z[:, j]) <= 100(1-majority[j]))
end

@objective(m, Max, sum(majority))

status = solve(m)
println(getvalue(majority))
println(getobjectivevalue(m))
zz = getvalue(z)

Academic license - for non-commercial use only
[1.0, 1.0, 1.0, 0.0, 0.0]
3.0


10×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0

Therefore, according to the solver:

district 1 includes: city 3, 4 and 8

district 2 includes: city 5

district 3 includes: city 6 and 9

district 4 includes: city 2 and 7

district 5 includes: city 1 and 10.

The governor has 3 Democratic majority districts.

2.The Queens problem

a.Queens not threaten each other

In [230]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

#for each queen, indicate their position on the board
#1:row num, #2:column num
@variable(m, z[1:8, 1:8], Bin)

#each row has less or equal one queen
for i in 1:8
    @constraint(m, sum(z[i, j] for j in 1:8) <= 1)
end

#each column has less than or equal one queen
for j in 1:8
    @constraint(m, sum(z[i, j] for i in 1:8) <= 1)
end

#above and including the main diagonal
for k in 0:6
    @constraint(m, sum(z[i, i+k] for i in 1:8-k) <= 1)
end

#below the main diagonal
for k in 1:6
    @constraint(m, sum(z[i+k, i] for i in 1:8-k) <= 1)
end

#above and including the anti diagonal
for k in 0:6
    @constraint(m, sum(z[i, 8-k-i+1] for i in 1:8-k) <= 1)
end

#below the anti diagonal
for k in 1:6
    @constraint(m, sum(z[i, 9+k-i] for i in 1+k:8) <= 1)
end

#has to be exactly 8 queens
@constraint(m, sum(z[i, j] for i in 1:8 for j in 1:8) == 8)

status = solve(m)
zza = (getvalue(z))

Academic license - for non-commercial use only


8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0

In [231]:
function print_board(zza)
println()
for i = 1:8
    for j = 1:8
        if zza[i, j] >0
            print_with_color(:black, "|")
            print_with_color(:red, "Q")
        else
            print("|", " ")
        end
    end
 print_with_color(:black, "|\n")
 println()
end
end

print_board(zza)


| | | | | |Q| | |

| | |Q| | | | | |

| | | | | | |Q| |

| |Q| | | | | | |

| | | |Q| | | | |

| | | | | | | |Q|

|Q| | | | | | | |

| | | | |Q| | | |



b.Queens not threaten each other with point symmetry

In [232]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

#for each queen, indicate their position on the board
#1:row num, #2:column num
@variable(m, z[1:8, 1:8], Bin)

#each row has less or equal one queen
for i in 1:8
    @constraint(m, sum(z[i, j] for j in 1:8) <= 1)
end

#each column has less than or equal one queen
for j in 1:8
    @constraint(m, sum(z[i, j] for i in 1:8) <= 1)
end

#above and including the main diagonal
for k in 0:6
    @constraint(m, sum(z[i, i+k] for i in 1:8-k) <= 1)
end

#below the main diagonal
for k in 1:6
    @constraint(m, sum(z[i+k, i] for i in 1:8-k) <= 1)
end

#above and including the anti diagonal
for k in 0:6
    @constraint(m, sum(z[i, 8-k-i+1] for i in 1:8-k) <= 1)
end

#below the anti diagonal
for k in 1:6
    @constraint(m, sum(z[i, 9+k-i] for i in 1+k:8) <= 1)
end

#has to be exactly 8 queens
@constraint(m, sum(z[i, j] for i in 1:8 for j in 1:8) == 8)
                
for i in 1:8
    for j in 1:8
        @constraint(m, z[i, j] == z[8-i+1, 8-j+1])
    end
end
                
status = solve(m)
zzb = (getvalue(z))

Academic license - for non-commercial use only


8×8 Array{Float64,2}:
  0.0  -0.0  -0.0  -0.0  -0.0   1.0  -0.0   0.0
 -0.0   0.0  -0.0   1.0  -0.0  -0.0   0.0  -0.0
 -0.0  -0.0   0.0  -0.0  -0.0   0.0   1.0  -0.0
  1.0  -0.0  -0.0   0.0   0.0  -0.0  -0.0  -0.0
  0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0
  0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0
  0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0
  0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0

In [233]:
print_board(zzb)


| | | | | |Q| | |

| | | |Q| | | | |

| | | | | | |Q| |

|Q| | | | | | | |

| | | | | | | |Q|

| |Q| | | | | | |

| | | | |Q| | | |

| | |Q| | | | | |



c.Each cell is threatened by at lease one queen

In [234]:
function get_diagonal_sum(i, j)
    
    k_diagonal = abs(j-i)
    if j >= i
        s_diagonal = sum(z[i, i+k_diagonal] for i in 1:8-k_diagonal)
    else
        s_diagonal = sum(z[i+k_diagonal, i] for i in 1:8-k_diagonal)
    end
    
    k_anti_diagonal = i+j
    if k_anti_diagonal <= 9
        k_anti_diagonal = abs(k_anti_diagonal - 9)
        s_anti_diagonal = sum(z[i, 9-k_anti_diagonal-i] 
            for i in 1:8-k_anti_diagonal)
    else
        k_anti_diagonal = k_anti_diagonal - 9
        s_anti_diagonal = sum(z[i, 9+k_anti_diagonal-i] 
            for i in 1+k_anti_diagonal:8)
    end
    return s_diagonal, s_anti_diagonal
end

get_diagonal_sum (generic function with 1 method)

In [235]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

#for each queen, indicate their position on the board
#1:row num, #2:column num
@variable(m, z[1:8, 1:8], Bin)

for i in 1:8
    for j in 1:8
        s_diagonal, s_anti_diagonal = get_diagonal_sum(i, j)
        total_sum = sum(z[i, k] for k in 1:8) + sum(z[l, j] for l in 1:8)  + 
        s_diagonal + s_anti_diagonal - 3*z[i, j]
        @constraint(m, total_sum >= 1)
    end
end

@objective(m, Min, sum(z))

status = solve(m)
println(getobjectivevalue(m))
zzc = getvalue(z)

Academic license - for non-commercial use only
5.0


8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [236]:
print_board(zzc)


| | | | | | | | |

| |Q| | | | | | |

| | | | | | | | |

| | | | | | |Q| |

| | | | | | | | |

|Q| | | | | | | |

| | | | |Q| | | |

|Q| | | | | | | |



We need 5 queens in order to threaten each cell at least once.

d.Each cell is threatened by at least one queen with point symmetry

In [237]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))

#for each queen, indicate their position on the board
#1:row num, #2:column num
@variable(m, z[1:8, 1:8], Bin)

for i in 1:8
    for j in 1:8
        s_diagonal, s_anti_diagonal = get_diagonal_sum(i, j)
        total_sum = sum(z[i, k] for k in 1:8) + sum(z[l, j] for l in 1:8)  + 
        s_diagonal + s_anti_diagonal - 3*z[i, j]
        @constraint(m, total_sum >= 1)
    end
end

for i in 1:8
    for j in 1:8
        @constraint(m, z[i, j] == z[8-i+1, 8-j+1])
    end
end

@objective(m, Min, sum(z))

status = solve(m)
println(getobjectivevalue(m))
zzd = getvalue(z)

Academic license - for non-commercial use only
6.0


8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [238]:
print_board(zzd)


| | | | | | | | |

| |Q| | | | | | |

| | | | | |Q| | |

| | | |Q| | | | |

| | | | |Q| | | |

| | |Q| | | | | |

| | | | | | |Q| |

| | | | | | | | |



We need 6 queens in this case. The minimum number of queens required changed from 5 to 6.

3.Paint production

In [239]:
using JuMP, Gurobi, NamedArrays, Cbc

A = [0 11 7 13 11
     5 0 13 15 15
     13 15 0 23 11
     9 13 5 0 3
     3 7 7 7 0]

m = Model(solver=GurobiSolver(OutputFlag=0))

# must formulate as IP this time
@variable(m, x[1:5, 1:5], Bin)
# one out-edge
@constraint(m, c1[j in 1:5], sum(x[i,j] for i in 1:5) == 1)
# one in-edge
@constraint(m, c2[i in 1:5], sum(x[i,j] for j in 1:5) == 1)
# no self-loops
@constraint(m, c3[i in 1:5], x[i,i] == 0 )

# MTZ variables and constraints
@variable(m, 1 <= u[1:5] <= 5)
@constraint(m, c4[i in 1:5, j in 2:5], u[i] - u[j] + N*x[i,j] <= 5-1 )

# minimize total cost
@objective(m, Min, sum(x .* A))

status = solve(m)
xx = getvalue(x)

Academic license - for non-commercial use only


5×5 Array{Float64,2}:
 -0.0  -0.0   0.0   1.0  -0.0
  1.0  -0.0  -0.0  -0.0  -0.0
 -0.0   0.0  -0.0  -0.0   1.0
 -0.0  -0.0   1.0  -0.0   0.0
  0.0   1.0  -0.0   0.0  -0.0

In [240]:
n = 5
println("The shortest possible cleaning time = ", getobjectivevalue(m), "min")
println("One possible optimal order of paint batches: ")
order = zeros(n)
for i = 1:n
 index = Int8(round(getvalue(u[i])))
 order[index] = i
end
order_string = ""
for i = 1:n
 if i == n
 order_string = string(order_string, order[i])
 else
 order_string = string(order_string, order[i], " => ")
 end
end
println(order_string)

The shortest possible cleaning time = 41.0min
One possible optimal order of paint batches: 
1.0 => 4.0 => 3.0 => 5.0 => 2.0


Therefore, the corresponding order of the paint batches is 1 -> 4 -> 3 -> 5 -> 2, this will give us a total washing time of 41 minutes.